The primes $3$, $7$, $109$, and $673$, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking $7$ and $109$, both $7109$ and $1097$ are prime. The sum of these four primes, $792$, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [1]:
import functools
from itertools import combinations, product, takewhile
from math import sqrt
from time import perf_counter

In [2]:
def primes_gen():
    primos = set()
    primos.add(2)
    yield 2

    valor = 3
    while True:
        validador = True
        for p in primos:
            if valor%p == 0:
                validador = False
                break

        if validador:
            primos.add(valor)
            yield valor

        valor += 2

In [3]:
def test_prime(num: int) -> bool:
    global primes, all_primes
    max_test = int(sqrt(num))
    while max(all_primes) < max_test:
        all_primes.append(next(primes))
    
    for n in takewhile(lambda x: x <= max_test, all_primes):
        if num % n == 0:
            return False

    return True

In [4]:
@functools.cache
def concat_n_test(i: int, j: int) -> bool:
    conc1 = int(str(i) + str(j))
    conc2 = int(str(j) + str(i))
    return test_prime(conc1) and test_prime(conc2)

In [24]:
def search(previous: list = [], max_size: int = 5, sumlimit: int = 100000):
    global primes_for
    
    if len(previous) == max_size:
        # print(previous)
        return (sum(previous), previous.copy())
    else:
        start_index = 0 if (len(previous) == 0) else (primes_for.index(previous[-1]) + 1 )
        for pr in primes_for[start_index:]:
            if pr >= sumlimit:
                break
            if all((concat_n_test(pr, j) for j in previous)):
                previous.append(pr)
                if len(previous) == 5:
                    pass
                result = search(previous=previous, max_size=max_size, sumlimit=sumlimit-pr)
                previous.pop()
                if result is not None:
                    return result
        return None

In [25]:
all_primes = []
primes = primes_gen()
while True:
    current = next(primes)
    all_primes.append(current)
    if current > 100000:
        break

primes_for = all_primes.copy()
primes_for.remove(2)
min_sum = 10**6
while True:

    result = search(sumlimit=min_sum-1)
    if not result:
        break
    print(result)
    if not min_sum:
        min_sum = result[0]
    elif result[0] < min_sum:
        answer = result
        min_sum = result[0]

print(f'The answer is {answer[0]}')

(98003, [3, 3119, 9887, 36263, 48731])
(76501, [3, 5323, 10357, 29587, 31231])
(34427, [7, 1237, 2341, 12409, 18433])
(26033, [13, 5197, 5701, 6733, 8389])
The answer is (26033, [13, 5197, 5701, 6733, 8389])


In [ ]:
# cópia de segurança do que eu estava criando.
def search(previous: list = [], max_size: int = 5, sumlimit: int=1000):
    global all_primes
    
    if previous:
        ini = all_primes.index(previous[-1]) + 1
    else:
        result = None
        for start in all_primes[1:]:
            result = search([start], max_size)
            if len(result) == max_size:
                return result
        else:
            raise Exception('Could not find a solution.')
    for prime in all_primes[ini:]:
        test = True
        for pre in previous:
            conc1 = int(str(pre) + str(prime))
            conc2 = int(str(prime) + str(pre))
            if not test_prime(conc1) or not test_prime(conc2):
                test = False
                break

        if test:
            previous.append(prime)
            if len(previous) == max_size:
                return previous
            else:
                return search(previous, max_size)
                

In [ ]:
print(result)

In [ ]:
all_primes = []
primes = primes_gen()
while True:
    current = next(primes)
    all_primes.append(current)
    if current > 10000:
        break

primes_minus_2 = all_primes.copy()
primes_minus_2.remove(2)


In [ ]:
print(f'{len(all_primes)=}; {max(all_primes)}')

In [ ]:
bad_pairs = set()
set_primes = combinations(all_primes, 2)
for p1, p2 in set_primes:
    conc1 = int(str(p1) + str(p2))
    conc2 = int(str(p2) + str(p1))
    if test_prime(conc1) and test_prime(conc2):
        # print(f'{p1}, {p2} is not bad')
        continue
    bad_pairs.add((p1, p2))
    bad_pairs.add((p2, p1))
    
len(bad_pairs)

In [ ]:
max(all_primes)

In [ ]:
set_primes = combinations(primes_minus_2, 4)
count = 0
set_sum = 0
start = perf_counter()
pos_results = []
for conj in set_primes:
    count += 1
    # if count % 1000000 == 0:
    #     print(f'{count=}; {conj=}; time elapsed: {perf_counter()-start}s')
    list_primes = list(conj)
    if 2 in conj: # 2 cannot be part of the set
        continue
    if sum(conj)>20000: # maximum sum allowed 
        continue
    if conj == (3, 7, 109, 673):
        continue
    list_primes.sort()
    good_set = True

    for p1, p2 in product(list_primes, repeat=2):
        if p1 == p2:
            continue
        
        if ((p1, p2) in bad_pairs) or ((p2, p1) in bad_pairs):
            good_set = False
            break

        # conc1 = int(str(p1)+str(p2))
        # conc2 = int(str(p2)+str(p1))

        # if not test_prime(conc1) or not test_prime(conc2):
        #     good_set = False
        #     bad_pairs.append((p1, p2))
        #     break

    if good_set:
        # set_sum = sum(conj)
        # break
        pos_results.append(conj)

# if good_set:
#     print(f'Resultado positivo. O conjunto {conj} tem soma {set_sum}')
# else:
#     print(f'Resultado negativo. Foram testados {count} conjuntos e o último foi {conj}.')
print(pos_results)
print(f'Tempo total: {perf_counter()-start}s')

In [ ]:
# print(pos_results)
len(pos_results)

In [ ]:
primes_minus_2 = all_primes.copy()
primes_minus_2.remove(2)

In [ ]:
set_primes = combinations(primes_minus_2, 4)
count = 0
set_sum = 0
start = perf_counter()
pos_results = []
for conj in set_primes:
    count += 1
    # if count % 1000000 == 0:
    #     print(f'{count=}; {conj=}; time elapsed: {perf_counter()-start}s')
    list_primes = list(conj)
    if 2 in conj: # 2 cannot be part of the set
        continue
    if sum(conj)>20000: # maximum sum allowed 
        continue
    if conj == (3, 7, 109, 673):
        continue
    list_primes.sort()
    good_set = True

    for p1, p2 in product(list_primes, repeat=2):
        if p1 == p2:
            continue
        
        if ((p1, p2) in bad_pairs) or ((p2, p1) in bad_pairs):
            good_set = False
            break

        # conc1 = int(str(p1)+str(p2))
        # conc2 = int(str(p2)+str(p1))

        # if not test_prime(conc1) or not test_prime(conc2):
        #     good_set = False
        #     bad_pairs.append((p1, p2))
        #     break

    if good_set:
        print(f'Testando o conjunto {conj}')
        for p5 in primes_minus_2:
            if p5 <= max(conj):
                continue
            list_primes = list(conj)
            list_primes.append(p5)
            conj = tuple(list_primes)
            
            for p1, p2 in product(list_primes, repeat=2):
                if p1 == p2:
                    continue
                
                if ((p1, p2) in bad_pairs) or ((p2, p1) in bad_pairs):
                    good_set = False
                    break

                conc1 = int(str(p1)+str(p2))
                conc2 = int(str(p2)+str(p1))

                if not test_prime(conc1) or not test_prime(conc2):
                    good_set = False
                    bad_pairs.add((p1, p2))
                    break

            if good_set:
                # conj = (p1, p2, p3, p4, p5)
                set_sum = sum(conj)
                break



if good_set:
    print(f'Resultado positivo. O conjunto {conj} tem soma {set_sum}')
else:
    print(f'Resultado negativo. Foram testados {count} conjuntos e o último foi {conj}.')
# print(pos_results)
print(f'Tempo total: {perf_counter()-start}s')

In [ ]:
# p1, p2, p3, p4 = (3, 7, 109, 673)
result4 = (3, 7, 109, 673)
for p5 in primes_minus_2:
    if p5 <= max(result4):
        continue
    list_primes = list(result4)
    list_primes.append(p5)
    conj = tuple(list_primes)
    good_set = True
    if p5 == 10009:
        pass
    
    for p1, p2 in product(list_primes, repeat=2):
        if p1 == p2:
            continue
        
        if ((p1, p2) in bad_pairs) or ((p2, p1) in bad_pairs):
            good_set = False
            break

        conc1 = int(str(p1)+str(p2))
        conc2 = int(str(p2)+str(p1))

        if not test_prime(conc1) or not test_prime(conc2):
            good_set = False
            bad_pairs.add((p1, p2))
            break

    if good_set:
        # conj = (p1, p2, p3, p4, p5)
        set_sum = sum(conj)
        break

if good_set:
    print(f'Resultado positivo. O conjunto {conj} tem soma {set_sum}')
else:
    print(f'Resultado negativo. Foram testados {count} conjuntos e o último foi {conj}.')

In [ ]:
if good_set:
    print(f'Resultado positivo. O conjunto {conj} tem soma {set_sum}')
else:
    print(f'Resultado negativo. Foram testados {count} conjuntos e o último foi {conj}.')